In [36]:
import pandas as pd
import os 
os.chdir(r'./')

In [37]:
df_Votes = pd.read_csv('BAF-al_2020.csv',dtype={'GEOID20_left':str,'COUNTYFP20':str})
df_CD = pd.read_csv('AL_94_CD116_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})

In [38]:
WhiPop = 'P0030003'
TarPop = 'P0030004'
TotPop = 'P0030001'
df_Votes.head()

,GEOID20_left,STATEFP20,COUNTYFP20,VTDST20,GEOID20_right,NAME20,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREOWRI,...,G20SACOWRI,G20SACRFRI,G20SACOWR2,G20SCCRWIN,G20SCCOWRI,G20SCCRKEL,G20SCCOWR2,G20PSCRCAV,G20PSCDCAS,G20PSCOWRI
0,010010201001000,1,001,000016,01001000016,Doster Comm Ctr,3746,1659,70,23,...,88,3943,81,3957,72,3945,74,3662,1693,9
1,010010201001001,1,001,000016,01001000016,Doster Comm Ctr,3746,1659,70,23,...,88,3943,81,3957,72,3945,74,3662,1693,9
2,010010201001002,1,001,000016,01001000016,Doster Comm Ctr,3746,1659,70,23,...,88,3943,81,3957,72,3945,74,3662,1693,9
3,010010201001003,1,001,000016,01001000016,Doster Comm Ctr,3746,1659,70,23,...,88,3943,81,3957,72,3945,74,3662,1693,9
4,010010201001004,1,001,000016,01001000016,Doster Comm Ctr,3746,1659,70,23,...,88,3943,81,3957,72,3945,74,3662,1693,9


In [39]:
df_CD.head()

,GEOCODE,COUNTY,VTD,CD116,SLDU18,SLDL18,POP100,HU100,INTPTLAT,INTPTLON,...,P0050001,P0050002,P0050003,P0050004,P0050005,P0050006,P0050007,P0050008,P0050009,P0050010
0,10239567001000,023,000006,07,24,65,30,22,32.229782,-88.208211,...,0,0,0,0,0,0,0,0,0,0
1,10239567001001,023,000006,07,24,65,53,35,32.235793,-88.250924,...,0,0,0,0,0,0,0,0,0,0
2,10239567001002,023,000006,07,24,65,32,23,32.238409,-88.266784,...,0,0,0,0,0,0,0,0,0,0
3,10239567001003,023,000006,07,24,65,0,0,32.235749,-88.276752,...,0,0,0,0,0,0,0,0,0,0
4,10239567001004,023,000006,07,24,65,5,1,32.231833,-88.282821,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df_Votes = df_Votes[['VTDST20','COUNTYFP20','G20PRERTRU','G20PREDBID']]
df_Votes = df_Votes.drop_duplicates().reset_index(drop=True)
df_Votes = df_Votes.rename(columns={'VTDST20':'VTD','COUNTYFP20':'COUNTY'})
# P003 is Cenuses 18+ count for P001
df_CD = df_CD[['VTD','COUNTY',WhiPop,TarPop,TotPop]]
df_CD = df_CD.groupby(['VTD','COUNTY']).sum()
df = df_CD.merge(df_Votes, how='inner',on=['VTD','COUNTY'])
df = df.drop_duplicates()
df = df.drop(columns=['COUNTY'])
df_sum = df.groupby(['VTD']).sum().reset_index()
df = df[['VTD']].merge(df_sum, on='VTD')
df = df.drop_duplicates()
assert len(df.index) != 0
assert len(df_Votes.index) != 0

In [41]:
df['Target_Percentage'] = df[TarPop]/df[TotPop]
df['White_Percentage'] = df[WhiPop]/df[TotPop]
df['Group'] = None
df = df.reset_index(drop=True)

for i ,row in df.iterrows():
    if row['Target_Percentage'] >= .9:
        df['Group'].iloc[i] = 'T0%'
    if row['Target_Percentage'] < .9:
        df['Group'].iloc[i] = 'T60%'
    if row['Target_Percentage'] < .6:
        df['Group'].iloc[i] = 'N50%'
    if row['Target_Percentage'] <= .4:
        df['Group'].iloc[i] = 'W60%'
    if row['Target_Percentage'] <= .1:
        df['Group'].iloc[i] = 'W90%'

c:\Miniconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [42]:
df.head()

,VTD,P0030003,P0030004,P0030001,G20PRERTRU,G20PREDBID,Target_Percentage,White_Percentage,Group
0,000001,573,1530,2142,651,1616,0.714286,0.267507,T60%
1,000002,608,1703,2365,345,834,0.720085,0.257082,T60%
2,000003,1165,2860,4121,962,1297,0.694006,0.282698,T60%
3,000004,2047,4224,6449,949,2159,0.654985,0.317414,T60%
4,000005,175,422,615,161,471,0.686179,0.284553,T60%


In [43]:
Tar_df = df[['VTD','Target_Percentage','G20PREDBID','G20PRERTRU',TotPop, TarPop]][(df['Group'] == 'T60%') | (df['Group'] == 'T90%')]
Whi_df = df[['VTD','White_Percentage','G20PREDBID','G20PRERTRU',TotPop, WhiPop]][(df['Group'] == 'W60%') | (df['Group'] == 'W90%')]
Tar_df['G20PRETOTAL'] = Tar_df['G20PREDBID'] + Tar_df['G20PRERTRU']
Whi_df['G20PRETOTAL'] = Whi_df['G20PREDBID'] + Whi_df['G20PRERTRU']

In [45]:
Tar_df['G20PREDBID_Percentage'] = Tar_df['G20PREDBID'] /Tar_df['G20PRETOTAL']
Tar_df['G20PRERTRU_Percentage'] = Tar_df['G20PRERTRU'] /Tar_df['G20PRETOTAL']
Whi_df['G20PREDBID_Percentage'] = Tar_df['G20PREDBID'] /Tar_df['G20PRETOTAL']
Whi_df['G20PRERTRU_Percentage'] = Tar_df['G20PRERTRU'] /Tar_df['G20PRETOTAL']
Tar_df['TurnOut_Percentage'] = Tar_df[TarPop]/Tar_df['G20PRETOTAL']
Whi_df['TurnOut_Percentage'] = Whi_df[WhiPop]/Whi_df['G20PRETOTAL']

In [46]:
Tar_df.head()

,VTD,Target_Percentage,G20PREDBID,G20PRERTRU,P0030001,P0030004,G20PRETOTAL,G20PREDBID_Percentage,G20PRERTRU_Percentage,TurnOut_Percentage
0,000001,0.714286,1616,651,2142,1530,2267,0.712836,0.287164,0.674901
1,000002,0.720085,834,345,2365,1703,1179,0.707379,0.292621,1.444444
2,000003,0.694006,1297,962,4121,2860,2259,0.574148,0.425852,1.266047
3,000004,0.654985,2159,949,6449,4224,3108,0.694659,0.305341,1.359073
4,000005,0.686179,471,161,615,422,632,0.745253,0.254747,0.667722
